In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from PIL import Image
import numpy as np
import sys
import subprocess
import gc

# try:
#     subprocess.check_call([sys.executable, "-m", "pip", "install", "sentencepiece"])
#     logger.info("Successfully installed sentencepiece")
# except Exception as e:
#     logger.error(f"Failed to install sentencepiece: {e}")
#     sys.exit(1)

# # Verify sentencepiece installation
# try:
#     import sentencepiece
#     logger.info(f"sentencepiece version: {sentencepiece.__version__}")
# except ImportError:
#     logger.error("Failed to import sentencepiece after installation")
#     sys.exit(1)
import sentencepiece
# Now import transformers after sentencepiece is installed

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor, AutoModelForVision2Seq, AutoModel, SiglipModel, SiglipProcessor
from transformers import get_scheduler
from tqdm import tqdm
import argparse
import random

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor, AutoModelForVision2Seq, AutoModel, SiglipModel, SiglipProcessor
from transformers import get_scheduler
from tqdm import tqdm
import argparse
import random
import logging
from pathlib import Path

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("siglip_training.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Set random seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Custom dataset for CIFAR10 image-answer pairs
class CIFAR10QADataset(Dataset):
    def __init__(self, csv_file, image_dir, tokenizer, processor, max_length=512):
        """
        Initialize the dataset.

        Args:
            csv_file (str): Path to the CSV file containing image-answer pairs
            image_dir (str): Directory containing the images
            tokenizer: Tokenizer for text processing
            processor: Processor for image processing
            max_length (int): Maximum sequence length for text
        """
        # Verify image directory exists
        if not os.path.exists(image_dir):
            raise ValueError(f"Image directory {image_dir} does not exist")

        # Load and verify CSV file
        try:
            self.data = pd.read_csv(csv_file)
            self.data['len'] = self.data['Answer'].apply(lambda x: len(x))
            self.data = self.data[self.data['len']>10]
            del self.data['len']


            self.data.columns = ['image_path', 'question', 'answer']
            logger.info(f"CSV columns: {self.data.columns.tolist()}")
            required_columns = ['image_path', 'question', 'answer']
            missing_columns = [col for col in required_columns if col not in self.data.columns]
            if missing_columns:
                raise ValueError(f"CSV file is missing required columns: {missing_columns}")
        except Exception as e:
            raise ValueError(f"Error loading CSV file {csv_file}: {str(e)}")

        self.image_dir = image_dir
        self.tokenizer = tokenizer
        self.processor = processor
        self.max_length = max_length

        # Log dataset statistics
        logger.info(f"Loaded {len(self.data)} image-answer pairs from {csv_file}")
        logger.info(f"Image directory: {image_dir}")

        # Verify first few images can be loaded
        self._verify_images()

    def _verify_images(self, num_samples=5):
        """Verify that images can be loaded correctly."""
        sample_indices = np.random.choice(len(self.data), min(num_samples, len(self.data)), replace=False)
        for idx in sample_indices:
            try:
                row = self.data.iloc[idx]
                image_path = os.path.join(self.image_dir, row['image_path'])
                logger.info(f"Verifying image: {image_path}")

                if not os.path.exists(image_path):
                    raise FileNotFoundError(f"Image file not found: {image_path}")

                image = Image.open(image_path).convert('RGB')
                # Try processing the image
                processed = self.processor(images=image, return_tensors="pt")
                logger.info(f"Successfully verified image: {image_path}")
                logger.info(f"Processed image shape: {processed.pixel_values.shape}")
            except Exception as e:
                logger.error(f"Error verifying image at index {idx}: {str(e)}")
                raise

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        try:
            # Get row data
            row = self.data.iloc[idx]
            logger.info(f"Processing item {idx}: {row['image_path']}")

            # Construct image path
            image_path = os.path.join(self.image_dir, row['image_path'])
            if not os.path.exists(image_path):
                raise FileNotFoundError(f"Image file not found: {image_path}")

            # Load and process image
            image = Image.open(image_path).convert('RGB')
            image_inputs = self.processor(images=image, return_tensors="pt")

            # Verify image inputs
            if image_inputs.pixel_values is None:
                raise ValueError(f"Failed to process image: {image_path}")

            logger.info(f"Processed image shape: {image_inputs.pixel_values.shape}")

            # Process text
            text = f"Question: {row['question']}\nAnswer: {row['answer']}"
            text_inputs = self.tokenizer(
                text,
                max_length=self.max_length,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )

            # Ensure attention_mask exists
            if 'attention_mask' not in text_inputs:
                input_ids = text_inputs.input_ids.squeeze(0)
                attention_mask = torch.ones_like(input_ids)
                attention_mask[input_ids == self.tokenizer.pad_token_id] = 0
            else:
                attention_mask = text_inputs.attention_mask.squeeze(0)

            # Create batch dictionary with correct shapes
            batch = {
                "pixel_values": image_inputs.pixel_values.squeeze(),  # Remove batch dimension from processor
                "input_ids": text_inputs.input_ids.squeeze(0),
                "attention_mask": attention_mask,
                "labels": text_inputs.input_ids.squeeze(0).clone()
            }

            # Verify batch contents
            for k, v in batch.items():
                if v is None:
                    raise ValueError(f"Batch item {k} is None")
                logger.info(f"Batch item {k} shape: {v.shape if hasattr(v, 'shape') else 'no shape'}")

            return batch

        except Exception as e:
            logger.error(f"Error loading item at index {idx}: {str(e)}")
            raise

# SigLiP model definition
class SigLiPModel(nn.Module):
    def __init__(self, vision_model, text_model, projection_dim=512):
        super().__init__()
        self.vision_model = vision_model
        self.text_model = text_model

        # Freeze the text model (Phi3)
        for param in self.text_model.parameters():
            param.requires_grad = False

        # Get the correct hidden size for vision model
        # SigLiP models use 'hidden_size' or 'vision_hidden_size'
        vision_hidden_size = getattr(vision_model.config, 'hidden_size',
                                    getattr(vision_model.config, 'vision_hidden_size', 768))

        # Vision encoder projection
        self.vision_projection = nn.Sequential(
            nn.Linear(vision_hidden_size, projection_dim),
            nn.LayerNorm(projection_dim),
            nn.GELU(),
            nn.Linear(projection_dim, projection_dim)
        )

        # Get the correct hidden size for text model
        # Phi3 models use 'hidden_size'
        text_hidden_size = getattr(text_model.config, 'hidden_size', 2048)

        # Text encoder projection
        self.text_projection = nn.Sequential(
            nn.Linear(text_hidden_size, projection_dim),
            nn.LayerNorm(projection_dim),
            nn.GELU(),
            nn.Linear(projection_dim, projection_dim)
        )

        # Temperature parameter for contrastive loss
        self.temperature = nn.Parameter(torch.ones([]) * 0.07)

        # Cross-attention for fusion
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=projection_dim,
            num_heads=8,
            dropout=0.1
        )

        # Get the correct vocab size for text model
        vocab_size = getattr(text_model.config, 'vocab_size', 50257)

        # Final projection for generation
        self.final_projection = nn.Linear(projection_dim, vocab_size)

    def forward(self, pixel_values, input_ids, attention_mask, labels=None):
        batch_size = pixel_values.size(0)

        # Ensure pixel_values has the correct shape [batch_size, channels, height, width]
        if pixel_values.dim() == 3:
            pixel_values = pixel_values.unsqueeze(0)

        # Get vision features
        try:
            # Process each image in the batch individually to avoid memory issues
            vision_features_list = []
            for i in range(batch_size):
                # Extract single image
                single_image = pixel_values[i:i+1]  # Keep batch dimension

                # Process with vision model
                vision_outputs = self.vision_model(single_image)

                # Extract features
                if hasattr(vision_outputs, 'last_hidden_state'):
                    features = vision_outputs.last_hidden_state[:, 0, :]  # Use [CLS] token
                elif hasattr(vision_outputs, 'pooler_output'):
                    features = vision_outputs.pooler_output
                else:
                    # Fallback to using the entire output
                    features = vision_outputs[0][:, 0, :] if isinstance(vision_outputs, tuple) else vision_outputs[:, 0, :]

                vision_features_list.append(features)

            # Concatenate features
            vision_features = torch.cat(vision_features_list, dim=0)

        except Exception as e:
            logger.error(f"Error in vision model forward pass: {str(e)}")
            logger.error(f"pixel_values shape: {pixel_values.shape}")
            raise

        # Project vision features
        vision_embeddings = self.vision_projection(vision_features)

        # Get text features
        text_outputs = self.text_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )

        # Extract text features
        if hasattr(text_outputs, 'hidden_states') and text_outputs.hidden_states:
            text_features = text_outputs.hidden_states[-1][:, 0, :]  # Use [CLS] token
        elif hasattr(text_outputs, 'last_hidden_state'):
            text_features = text_outputs.last_hidden_state[:, 0, :]  # Use [CLS] token
        else:
            # Fallback to using the entire output
            text_features = text_outputs[0][:, 0, :] if isinstance(text_outputs, tuple) else text_outputs[:, 0, :]

        # Project text features
        text_embeddings = self.text_projection(text_features)

        # Cross-attention fusion
        vision_embeddings = vision_embeddings.unsqueeze(1)  # Add sequence dimension
        text_embeddings = text_embeddings.unsqueeze(1)  # Add sequence dimension

        # Ensure shapes are correct for cross-attention
        if vision_embeddings.size(0) != text_embeddings.size(0):
            # Handle batch size mismatch
            vision_embeddings = vision_embeddings.expand(text_embeddings.size(0), -1, -1)

        fused_features, _ = self.cross_attention(
            query=text_embeddings,
            key=vision_embeddings,
            value=vision_embeddings
        )

        # Project to vocabulary
        logits = self.final_projection(fused_features.squeeze(1))

        # Calculate loss if labels are provided
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            # Reshape logits and labels for loss calculation
            logits = logits.view(-1, logits.size(-1))
            labels = labels.view(-1)
            loss = loss_fct(logits, labels)

        # Ensure all outputs have the correct shape
        vision_embeddings = vision_embeddings.squeeze(1)  # Remove sequence dimension
        text_embeddings = text_embeddings.squeeze(1)  # Remove sequence dimension

        return {
            "loss": loss if loss is not None else torch.tensor(0.0, device=logits.device),
            "logits": logits,
            "vision_embeddings": vision_embeddings,
            "text_embeddings": text_embeddings
        }

# Training function
def train(args):
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")

    # Load models and tokenizers
    logger.info("Loading models and tokenizers...")

    # Load vision model (SigLiP)
    vision_processor = AutoProcessor.from_pretrained(args.vision_model_name)
    vision_model = AutoModelForVision2Seq.from_pretrained(args.vision_model_name)

    # Load text model (Phi3)
    text_tokenizer = AutoTokenizer.from_pretrained(args.text_model_name)
    text_model = AutoModelForCausalLM.from_pretrained(args.text_model_name)

    # Create dataset and dataloader
    logger.info("Creating dataset and dataloader...")
    dataset = CIFAR10QADataset(
        csv_file=args.csv_file,
        image_dir=args.image_dir,
        tokenizer=text_tokenizer,
        processor=vision_processor,
        max_length=args.max_length
    )

    dataloader = DataLoader(
        dataset,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=args.num_workers
    )

    # Create SigLiP model
    logger.info("Creating SigLiP model...")
    model = SigLiPModel(
        vision_model=vision_model,
        text_model=text_model,
        projection_dim=args.projection_dim
    )
    model.to(device)

    # Create optimizer
    optimizer = optim.AdamW(
        model.parameters(),
        lr=args.learning_rate,
        weight_decay=args.weight_decay
    )

    # Create learning rate scheduler
    num_training_steps = len(dataloader) * args.num_epochs
    lr_scheduler = get_scheduler(
        name="cosine",
        optimizer=optimizer,
        num_warmup_steps=args.warmup_steps,
        num_training_steps=num_training_steps
    )

    # Training loop
    logger.info("Starting training...")
    model.train()
    global_step = 0

    for epoch in range(args.num_epochs):
        epoch_loss = 0.0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{args.num_epochs}")

        for batch in progress_bar:
            # Move batch to device
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Forward pass
            outputs = model(
                pixel_values=pixel_values,
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs["loss"]

            # Backward pass
            loss.backward()

            # Clip gradients
            if args.max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

            # Update weights
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            # Update progress bar
            epoch_loss += loss.item()
            progress_bar.set_postfix({"loss": loss.item()})

            global_step += 1

        # Log epoch metrics
        avg_epoch_loss = epoch_loss / len(dataloader)
        logger.info(f"Epoch {epoch+1}/{args.num_epochs}, Average Loss: {avg_epoch_loss:.4f}")

        # Save checkpoint
        if (epoch + 1) % args.save_steps == 0:
            checkpoint_dir = Path(args.output_dir) / f"checkpoint-{epoch+1}"
            checkpoint_dir.mkdir(parents=True, exist_ok=True)

            model.save_pretrained(checkpoint_dir)
            vision_processor.save_pretrained(checkpoint_dir)
            text_tokenizer.save_pretrained(checkpoint_dir)

            logger.info(f"Saved checkpoint to {checkpoint_dir}")

    # Save final model
    logger.info(f"Saving final model to {args.output_dir}")
    model.save_pretrained(args.output_dir)
    vision_processor.save_pretrained(args.output_dir)
    text_tokenizer.save_pretrained(args.output_dir)

# Evaluation function
def evaluate(args):
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")

    # Load models and tokenizers
    logger.info("Loading models and tokenizers...")

    # Load vision model (SigLiP)
    vision_processor = AutoProcessor.from_pretrained(args.vision_model_name)
    vision_model = AutoModelForVision2Seq.from_pretrained(args.vision_model_name)

    # Load text model (Phi3)
    text_tokenizer = AutoTokenizer.from_pretrained(args.text_model_name)
    text_model = AutoModelForCausalLM.from_pretrained(args.text_model_name)

    # Load trained SigLiP model
    model = SigLiPModel(
        vision_model=vision_model,
        text_model=text_model,
        projection_dim=args.projection_dim
    )
    model.load_state_dict(torch.load(os.path.join(args.output_dir, "pytorch_model.bin")))
    model.to(device)
    model.eval()

    # Create dataset and dataloader
    logger.info("Creating dataset and dataloader...")
    dataset = CIFAR10QADataset(
        csv_file=args.eval_csv_file,
        image_dir=args.image_dir,
        tokenizer=text_tokenizer,
        processor=vision_processor,
        max_length=args.max_length
    )

    dataloader = DataLoader(
        dataset,
        batch_size=args.eval_batch_size,
        shuffle=False,
        num_workers=args.num_workers
    )

    # Evaluation loop
    logger.info("Starting evaluation...")
    total_loss = 0.0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            # Move batch to device
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Forward pass
            outputs = model(
                pixel_values=pixel_values,
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs["loss"]
            logits = outputs["logits"]

            # Calculate metrics
            total_loss += loss.item()

            # Get predictions
            predictions = torch.argmax(logits, dim=-1)
            all_predictions.extend(predictions.cpu().numpy().tolist())
            all_labels.extend(labels.cpu().numpy().tolist())

    # Calculate average loss
    avg_loss = total_loss / len(dataloader)
    logger.info(f"Evaluation Loss: {avg_loss:.4f}")

    return avg_loss

# Main function
def main(csv_file, image_dir, eval_csv_file=None, output_dir="output",
         model_name="google/siglip-base-patch16-224", batch_size=32,
         num_epochs=10, learning_rate=5e-5, num_workers=4,
         device=None, seed=42, save_every=1, gradient_accumulation_steps=4):
    """
    Main function to train SigLiP model on CIFAR10 QA dataset.
    """
    # Set device if not provided
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # Set random seeds
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    # Create output directory
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Clear memory before loading models
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()

    # Load tokenizer and processor
    logger.info("Loading tokenizer and processor...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    processor = SiglipProcessor.from_pretrained(model_name)

    # Create datasets
    logger.info("Creating datasets...")
    train_dataset = CIFAR10QADataset(
        csv_file,
        image_dir,
        tokenizer,
        processor
    )

    eval_dataset = CIFAR10QADataset(
        eval_csv_file,
        image_dir,
        tokenizer,
        processor
    ) if eval_csv_file else None

    # Create data loaders with smaller batch size if needed
    logger.info("Creating data loaders...")
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=False  # Disable pin_memory to reduce memory usage
    )

    eval_loader = DataLoader(
        eval_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=False  # Disable pin_memory to reduce memory usage
    ) if eval_dataset else None

    # Clear memory before loading models
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()

    # Initialize model
    logger.info("Loading vision model (SigLiP)...")
    # Load vision model (SigLiP) with reduced precision
    vision_model = SiglipModel.from_pretrained(model_name, torch_dtype=torch.float16, trust_remote_code=True)

    # Clear memory before loading text model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()

    logger.info("Loading text model (Phi3)...")
    # Load text model (Phi3) with reduced precision
    text_model_name = "microsoft/phi-2"
    text_tokenizer = AutoTokenizer.from_pretrained(text_model_name)
    text_model = AutoModelForCausalLM.from_pretrained(text_model_name, torch_dtype=torch.float16)

    # Clear memory before creating combined model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()

    logger.info("Creating SigLiP model...")
    # Create SigLiP model with both vision and text models
    model = SigLiPModel(
        vision_model=vision_model,
        text_model=text_model,
        projection_dim=512
    )

    # Enable gradient checkpointing to save memory
    model.vision_model.gradient_checkpointing_enable()
    model.text_model.gradient_checkpointing_enable()

    # Move model to device in a memory-efficient way
    if torch.cuda.is_available():
        # Move model to device in chunks with error handling
        try:
            # First move the vision model
            model.vision_model = model.vision_model.to(device)
            torch.cuda.empty_cache()
            gc.collect()

            # Then move the text model
            model.text_model = model.text_model.to(device)
            torch.cuda.empty_cache()
            gc.collect()

            # Finally move the remaining components
            for name, param in model.named_parameters():
                if not name.startswith(('vision_model.', 'text_model.')):
                    param.data = param.data.to(device)
                    if param.grad is not None:
                        param.grad.data = param.grad.data.to(device)
                    torch.cuda.empty_cache()
                    gc.collect()
        except RuntimeError as e:
            logger.error(f"Error moving model to device: {str(e)}")
            logger.error("Trying to free up memory and retry...")
            torch.cuda.empty_cache()
            gc.collect()
            # Try again with a smaller batch size
            batch_size = batch_size // 2
            logger.info(f"Reduced batch size to {batch_size}")
            return main(csv_file, image_dir, eval_csv_file, output_dir,
                       model_name, batch_size, num_epochs, learning_rate,
                       num_workers, device, seed, save_every,
                       gradient_accumulation_steps)
    else:
        model = model.to(device)

    # Initialize optimizer and scheduler
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = len(train_loader) * num_epochs // gradient_accumulation_steps
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    # Training loop
    best_eval_loss = float('inf')
    for epoch in range(num_epochs):
        # Train
        model.train()
        train_loss = 0
        optimizer.zero_grad()  # Zero gradients at the start of epoch

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}")

        for i, batch in enumerate(progress_bar):
            try:
                # Move batch to device in a memory-efficient way
                for k, v in batch.items():
                    if v is None:
                        raise ValueError(f"Batch item {k} is None")
                    batch[k] = v.to(device)
                    logger.info(f"Batch item {k} shape: {v.shape if hasattr(v, 'shape') else 'no shape'}")

                # Forward pass
                outputs = model(**batch)
                if outputs["loss"] is None:
                    raise ValueError("Model returned None loss")
                loss = outputs["loss"] / gradient_accumulation_steps  # Scale loss for gradient accumulation
                loss.backward()

                # Update weights after accumulating gradients
                if (i + 1) % gradient_accumulation_steps == 0 or (i + 1) == len(train_loader):
                    optimizer.step()
                    lr_scheduler.step()
                    optimizer.zero_grad()

                train_loss += loss.item() * gradient_accumulation_steps
                progress_bar.set_postfix({"loss": loss.item() * gradient_accumulation_steps})

                # Clear cache more frequently to free up memory
                if i % 5 == 0 and torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    gc.collect()

                # Delete intermediate tensors to free memory
                del outputs
                del loss
                torch.cuda.empty_cache()

            except Exception as e:
                logger.error(f"Error processing batch {i}: {str(e)}")
                logger.error(f"Batch contents: {[(k, v.shape if hasattr(v, 'shape') else 'no shape') for k, v in batch.items()]}")
                raise

        avg_train_loss = train_loss / len(train_loader)
        logger.info(f"Epoch {epoch + 1} - Average training loss: {avg_train_loss:.4f}")

        # Clear memory after training
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gc.collect()

        # Evaluate
        if eval_loader:
            model.eval()
            eval_loss = 0

            with torch.no_grad():
                for batch in eval_loader:
                    try:
                        # Move batch to device in a memory-efficient way
                        for k, v in batch.items():
                            if v is None:
                                raise ValueError(f"Batch item {k} is None")
                            batch[k] = v.to(device)

                        outputs = model(**batch)
                        if outputs["loss"] is None:
                            raise ValueError("Model returned None loss")
                        eval_loss += outputs["loss"].item()

                        # Delete intermediate tensors to free memory
                        del outputs

                        # Clear cache more frequently to free up memory
                        if torch.cuda.is_available():
                            torch.cuda.empty_cache()
                            gc.collect()

                    except Exception as e:
                        logger.error(f"Error processing evaluation batch: {str(e)}")
                        logger.error(f"Batch contents: {[(k, v.shape if hasattr(v, 'shape') else 'no shape') for k, v in batch.items()]}")
                        raise

            avg_eval_loss = eval_loss / len(eval_loader)
            logger.info(f"Epoch {epoch + 1} - Average evaluation loss: {avg_eval_loss:.4f}")

            # Save best model
            if avg_eval_loss < best_eval_loss:
                best_eval_loss = avg_eval_loss
                model.save_pretrained(output_dir / "best_model")
                tokenizer.save_pretrained(output_dir / "best_model")
                logger.info(f"Saved best model with evaluation loss: {best_eval_loss:.4f}")

        # Save checkpoint
        if (epoch + 1) % save_every == 0:
            checkpoint_dir = output_dir / f"checkpoint-{epoch + 1}"
            model.save_pretrained(checkpoint_dir)
            tokenizer.save_pretrained(checkpoint_dir)
            logger.info(f"Saved checkpoint at epoch {epoch + 1}")

        # Clear memory after epoch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gc.collect()

    logger.info("Training completed!")



# For command line usage
main(
    csv_file="cifar10_qa_results.csv",
    image_dir="cifar10_images",
    eval_csv_file="cifar10_qa_results.csv",
    output_dir="cifaroutput",
    batch_size=32,
    num_epochs=10,
    model_name = "google/siglip-base-patch16-224"
)